In [1]:
import os 
import time
import shutil
import requests
from glob import glob

import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt

import tifffile
import rasterio
import geopandas as gpd # geo 데이터프레임 만들기 위함
from pyidw import idw # 보간법 실행
from osgeo import gdal # 래스터 자르기 anaconda로 설치
from rasterio.plot import show # 래스터 시각화
from shapely.geometry import Point
from rasterio.features import geometry_mask
from rasterio.transform import from_origin

In [2]:
fire=pd.read_csv("fire.csv",encoding='utf8') #강원도 산불 데이터 
loc_data=pd.read_csv("loc_list.csv") # asos 지점과 경위도 데이터 

In [3]:
fire=fire[['field1', 'field2', '_X', '_Y']]
fire.columns=['date','time','lon','lat']
fire['time'] = fire['time'].apply(lambda x: str(x).zfill(6))
fire['input'] = fire['date'].astype(str)+fire['time'].apply(lambda x: str(x)[:2]).str.zfill(2).astype(str)
fire

,date,time,lon,lat,input
0,20110122,233500,128.869839,37.782030,2011012223
1,20110201,210700,128.852520,37.792612,2011020121
2,20110226,164200,127.955332,37.379769,2011022616
3,20110212,061800,127.879749,37.220708,2011021206
4,20110211,175000,128.128864,37.772593,2011021117
...,...,...,...,...,...
1048,20210115,153938,128.258791,38.075039,2021011515
1049,20210220,131300,127.548490,38.179367,2021022013
1050,20210417,093354,128.000383,37.659658,2021041709
1051,20210426,172913,127.954054,37.324059,2021042617


# 데이터 크롤링 

In [ ]:
# asos 데이터 크롤링, 서비스키만 바꿔서 사용하면 됨 
# 산불난 시간의 데이터를 크롤링하였음 
# 시간,습도,풍속,(풍향),강수량,온도

In [ ]:
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

def weather_crawling(date,locn):
    # api 호출시 매개변수 설정 
    params ={'serviceKey' : 'pXv2lwkQOlbcBekMvEjur8cGaDpQb84Upv6ZboITLKGudf2//PqPjL6QABwMtAhzNilIwndbgQdo9lBTUwy3XA==', #AuthenticationKey
             'pageNo' : '1',
             'numOfRows' : '10',
             'dataType' : 'JSON', 
             'dataCd' : 'ASOS', 
             'dateCd' : 'HR', 
             'startDt' : date[:8], #startdate
             'startHh' : date[8:], #starttime
             'endDt' : date[:8], # end date
             'endHh' : str(int(date[8:])+1), # end time
             'stnIds' : locn 
            }

    response = requests.get(url, params=params,verify=False)
    
    # 데이터가 있으면 불러오고 없으면 nan값을 넣음
    # 변수는 직접 추가해도 됨. 
    # 현재는 시간(데이터유무알기위해서),습도,풍속,(풍향),강수량,기온
    
    try:
        json_obj = json.loads(response.content)
        json_obj=json_obj["response"]["body"]["items"]["item"][0]
        time=json_obj['tm']
        humidity=json_obj['hm']
        windspeed=json_obj['ws']
        winddir=json_obj['wd']
        rain=json_obj['rn']
        temp=json_obj['ta']
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
    
    return time, humidity, windspeed, winddir, rain, temp

# asos지점, 경위도 저장 파일 불러옴 
loc_list=loc_data['지점번호']
result=[]
os.makedirs(f"data_set/origin_data", exist_ok=True) # 폴더 생성 
for i in tqdm(range(len(fire))):
    tmp=pd.DataFrame(columns=['num','location information', 'longitude','latitude','time','humidity','wind speed','wind direction','precipitation','temperature'])
    for j in range(len(loc_list)):
        new_data=[i,loc_list[j],loc_data['lon'][j],loc_data['lat'][j]]
        new_data.extend(weather_crawling(fire['input'][i],loc_list[j])) 
        tmp = pd.concat([tmp, pd.DataFrame([new_data], columns=tmp.columns)], ignore_index=True)
    tmp.to_csv(f"data_set/origin_data/data_{i}.csv",encoding='cp949',index=False)
for i in range(len(fire)):
    tmp=pd.read_csv(f"data_set/origin_data/data_{i}.csv",encoding='cp949')
    result.append(tmp)        
weather_data = pd.concat(result)
weather_data

In [11]:
# 크롤링한 데이터를 전부 합침 
weather_data.reset_index(inplace=True)
weather_data.to_csv(f"data_set/weather_data.csv",encoding='cp949',index=False)
weather_data

,index,num,location information,longitude,latitude,time,humidity,wind speed,wind direction,precipitation,temperature
0,0,0,90,128.56473,38.25085,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,93,127.75443,37.94738,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,95,127.30420,38.14787,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,98,127.06070,37.90188,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,99,126.76648,37.88589,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
102136,92,1052,288,128.74412,35.49147,2021-05-13 15:00,34.0,2.0,160.0,NaN,29.8
102137,93,1052,289,127.87910,35.41300,NaN,NaN,NaN,NaN,NaN,NaN
102138,94,1052,294,128.60459,34.88818,NaN,NaN,NaN,NaN,NaN,NaN
102139,95,1052,295,127.92641,34.81662,NaN,NaN,NaN,NaN,NaN,NaN


# 기상 데이터 보간 

In [10]:
# 각 산불(1053)개에 맞는 기상데이터들을 보간함
# 보간시 하나의 feature라도 없으면 전부 drop( 결측치 )
# 한 지점이라도 살아있으면 보간이 되서 우선 냅둠 
# 즉 어떤 산불은 지점 여러개로 보간된 데이터(정확도 높), 어떤건 지점한두개로 보간된 데이터(정확도 낮 )

In [24]:
%pwd

'C:\\Users\\ebdl\\산불예측\\산불예측'

csv 하나로 합치기~

In [25]:
path_notfire="data_set/origin_data/"
list_notfire=os.listdir(path_notfire)

weather_data=pd.read_csv(f"{path_notfire}data_0.csv",encoding='utf-8-sig')
for i in range(1,len(list_notfire)):
    data=pd.read_csv(f"{path_notfire}data_{i}.csv",encoding='utf-8-sig')
    weather_data=pd.concat([weather_data,data])

# 크롤링한 데이터를 전부 합침 
weather_data.reset_index(inplace=True)
weather_data.to_csv(f"data_set/weather_data.csv",encoding='cp949',index=False)


보간하기~

In [ ]:
features=['humidity','wind speed','rainfall','temp']

weather_data=pd.read_csv("data_set/weather_data.csv",encoding='cp949')
weather_data.columns=['index', 'num', 'location information', 'longitude', 'latitude', 'time','humidity', 'wind speed', 'wind direction', 'rainfall','temp']
weather_data.dropna(subset=['time'], inplace=True)
weather_data['rainfall']=weather_data['rainfall'].fillna(0)

for i in range(len(fire)):
    tmp=weather_data[weather_data['num']==i]
    tmp=tmp.dropna()
    # 결측치 전부 drop 후에 데이터가 없으면 보간하지 않음
    if(len(tmp)==0):
        print("There is no data")
        continue
    tmp.drop(['num','location information','time'],axis=1,inplace=True)
    tmp = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp.longitude, tmp.latitude))
    tmp.to_file(f'data{i}.shp')
    for j in range(len(features)):
        os.makedirs(f"data_set/tif_data/{features[j]}", exist_ok=True)
        idw.idw_interpolation(
            input_point_shapefile=f'data{i}.shp', # 보간하고자 하는 shp 파일 
            extent_shapefile="boundary/boundary.shp", # 경계 shp 파일(현재 강원도)
            column_name=features[j], # 보간하고자 하는 feature 이름. 
            power=2, # 거리 가중치 계수 
            search_radious=8, # 검색하고자 하는 범위 
            output_resolution=400, # 결과물 해상도 
        )
        shutil.move(f"data{i}_idw.tif", f"data_set/tif_data/{features[j]}/data{i}_idw.tif")
    os.remove(f"data{i}.shp")
    os.remove(f"data{i}.cpg")
    os.remove(f"data{i}.dbf")
    os.remove(f"data{i}.shx")

# 산불 발생 지역 tif 생성 

In [ ]:
# 산불 난 지점의 위치를 찾기위해서 각 산불난 위치 아래와 같은 순서로 진행
# 우선 산불난 지점을 shp로 찍음
# shp를 tif로 만들고 강원도 경계에 맞게 자름
# 만든 tif에서 산불 난 지점만 9999로 표시하고 나머진 32767로 표시되게 함( 강원도 지역구분은 안됨)

In [28]:
os.makedirs(f"data_set/fire_loc", exist_ok=True)
for i in range(len(fire)):
    tmp=pd.DataFrame(fire.iloc[i])
    tmp=tmp.T
    tmp = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp.lon, tmp.lat))
    tmp.drop(['date','time','lon','lat','input'],axis=1,inplace=True)
    tmp['fire']=1
    tmp.to_file('fire_data.shp')

    shp_path  = "fire_data.shp" # 현재 shp파일 이름 
    boundary_path='boundary/boundary.shp'
    tif_path  = "fire_data.tif" # 만들고자 하는 tif파일 이름

    driver = gdal.GetDriverByName('GTiff')

    # 산불 shp 파일을 읽어오기
    shp_datasource = gdal.OpenEx(shp_path, gdal.OF_VECTOR)
    
    # 경계shp 파일을 읽어오기
    boundary_datasource = gdal.OpenEx(boundary_path, gdal.OF_VECTOR)

    # tif 파일 생성
    tif_datasource = driver.Create(tif_path, 400, 278, 1, gdal.GDT_Float32)

    # 좌표계 설정
    tif_datasource.SetProjection(shp_datasource.GetProjection())
    
    
    # 강원도 경계값 가져옴 
    boundary = gpd.read_file(boundary_path)
    xmin, ymin, xmax, ymax = boundary.total_bounds

    # 강원도 경계값 가져온걸 위에서 설정한 400,278 즉 액셀 파일 크기형태로 설정 
    tif_datasource.SetGeoTransform((xmin, (xmax-xmin)/400, 0, ymax, 0, -(ymax-ymin)/278))

    band = tif_datasource.GetRasterBand(1)
    # 산불 난 지점 외에는 전부 32767 박기 
    band.SetNoDataValue(32767)
    # 산불 발생 위치 9999로 설정
    gdal.RasterizeLayer(tif_datasource, [1], shp_datasource.GetLayer(), burn_values=[9999]) 
    #gdal.RasterizeLayer(tif_datasource, [1], boundary_datasource.GetLayer(), burn_values=[32767])

    shp_datasource = None
    tif_datasource = None

    # file닫고 csv로 저장해야함
    tif_file = rasterio.open("fire_data.tif") 
    data = tifffile.imread('fire_data.tif')    
    data=pd.DataFrame(data)
    data.to_csv(f'data_set/fire_loc/fire_data{i}.csv')
    tif_file.close()
    
# 필요없는 파일 삭제
os.remove(f"fire_data.shp")
os.remove(f"fire_data.cpg")
os.remove(f"fire_data.dbf")
os.remove(f"fire_data.shx")
os.remove(f"fire_data.tif")

# 보간 파일 tif -> csv로 변환

In [29]:
path_dir = 'data_set/tif_data'
file_list=os.listdir(path_dir)

for i in range(len(file_list)):
    tif_list = sorted(glob(f"{path_dir}/{file_list[i]}/*.tif"))
    os.makedirs(f'data_set/csv_data/{file_list[i]}', exist_ok=True)
    for j in range(len(tif_list)):
        # 결측치 처리한다음에 파일명 맞춰주기 위해서 숫자 뽑음
        s_index = tif_list[j].rfind('\\')
        b_index = tif_list[j].rfind('.')
        image=rasterio.open(tif_list[j])
        image=pd.DataFrame(image.read(1))
        name=tif_list[j][s_index+5:b_index]
        image.to_csv(f"data_set/csv_data/{file_list[i]}/data{name}.csv",encoding='utf-8-sig')

In [30]:
tif_list = sorted(glob(f"{path_dir}/{file_list[0]}/*.tif"))
length=len(tif_list)

In [31]:
# dataframe에서 value 위치 찾기
def find_value_in_dataframe(df, value):
    res = []
    for i in df.index:
        for j in df.columns:
            if (df[j][i] == value):
                res.append([i, j])
    if (len(res) == 1):
        return (res[0])
    return (res)

# list 범위 제한
def handling_list_range(lst, lst_range):
    lst_res = []
    for i in (lst):
        if (i < lst_range[0]):
            lst_res.append(lst_range[0])
        elif (i >= lst_range[1]):
            lst_res.append(lst_range[1]-1)
        else:
            lst_res.append(i)
    return (lst_res)

# crop시 특정값 제외(강원도 밖 범위)
def handling_value_except(lst, v_except, center, radius):
    if (radius < 1):
        return
    # 동쪽 채우기
    for i in range(center[0]-radius, center[0]+radius+1):
        if (lst[i][center[1]-radius] == v_except):
            lst[i][center[1]-radius] = lst[i][center[1]-radius+1]
    # 서쪽 채우기
    for i in range(center[0]-radius, center[0]+radius+1):
        if (lst[i][center[1]+radius] == v_except):
            lst[i][center[1]+radius] = lst[i][center[1]+radius-1]
    # 남쪽 채우기
    for j in range(center[1]-radius, center[1]+radius+1):
        if (lst[center[0]+radius][j] == v_except):
            lst[center[0]+radius][j] = lst[center[0]+radius-1][j]
    # 북쪽 채우기
    for j in range(center[1]-radius, center[1]+radius+1):
        if (lst[center[0]-radius][j] == v_except):
            lst[center[0]-radius][j] = lst[center[0]-radius+1][j]
    # 꼭짓점 채우기
    if (lst[center[0]-radius][center[1]-radius] == v_except):
        lst[center[0]-radius][center[1]-radius] = lst[center[0]-radius][center[1]-radius+1]
    if (lst[center[0]-radius][center[1]+radius] == v_except):
        lst[center[0]-radius][center[1]+radius] = lst[center[0]-radius][center[1]+radius-1]
    if (lst[center[0]+radius][center[1]-radius] == v_except):
        lst[center[0]+radius][center[1]-radius] = lst[center[0]+radius][center[1]-radius+1]
    if (lst[center[0]+radius][center[1]+radius] == v_except):
        lst[center[0]+radius][center[1]+radius] = lst[center[0]+radius][center[1]+radius-1]

# dataframe에서 crop하기
def crop_dataframe_by_point(df, point, padding=0):
    # list index 구하기
    i_point = [list(df.index).index(point[0]), list(df.columns).index(point[1])]
    i_padding = [[i for i in range(i_point[0]-padding, i_point[0]+padding+1)], \
                 [i for i in range(i_point[1]-padding, i_point[1]+padding+1)]]
    
    # 범위 넘어가는 것 처리(끝값으로 처리)
    i_padding[0] = handling_list_range(i_padding[0], [0, df.shape[0]])
    i_padding[1] = handling_list_range(i_padding[1], [0, df.shape[1]])
    
    # index, column 구하기
    xy_padding = [[df.columns[x] for x in i_padding[1]], [df.index[y] for y in i_padding[0]]]
    
    # list 형태 뽑기
    res_lst = []
    for y in (xy_padding[1]):
        lst_i = []
        for x in (xy_padding[0]):
            lst_i.append(df[x][y])
        res_lst.append(lst_i)
    
    # 강원도 범위 처리
    for i in range(padding+1):
        handling_value_except(res_lst, 32767, [int(len(res_lst)/2), int(len(res_lst)/2)], i)
    
    return (np.array(res_lst, dtype=float))

def find_and_crop(df_find, df_crop, padding):
    df_find = pd.read_csv(df_find, index_col=0)
    df_crop = pd.read_csv(df_crop, index_col=0)
    df_crop.index = df_find.index
    df_crop.columns = df_find.columns
    return (crop_dataframe_by_point(df_crop, find_value_in_dataframe(df_find, 9999), padding))

In [14]:
%pwd

'C:\\Users\\ebdl\\산불예측\\산불예측'

# 보간된 csv 파일을 crop

## 7의 배수로 crop

In [22]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_hum'):
    os.makedirs('croped_fire_pred_hum')

# 원본 csv 파일 경로
csv_path = 'data_set/test_crop/hum_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_hum'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)
# pad_width = 3

# # 좌우로 0패딩
# padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# # 상하로 0패딩
# padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

#df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(0, n_rows-crop_size+1,7):
    for j in range(0, n_cols-crop_size+1,7):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1

In [24]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_rain'):
    os.makedirs('croped_fire_pred_rain')

# 원본 csv 파일 경로
csv_path = 'data_set/test_crop/rain_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_rain'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)
# pad_width = 3

# # 좌우로 0패딩
# padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# # 상하로 0패딩
# padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

#df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(0, n_rows-crop_size+1,7):
    for j in range(0, n_cols-crop_size+1,7):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1

In [25]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_temp'):
    os.makedirs('croped_fire_pred_temp')

# 원본 csv 파일 경로
csv_path = 'data_set/test_crop/temp_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_temp'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)
# pad_width = 3

# # 좌우로 0패딩
# padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# # 상하로 0패딩
# padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

#df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(0, n_rows-crop_size+1,7):
    for j in range(0, n_cols-crop_size+1,7):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1

In [26]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_wind'):
    os.makedirs('croped_fire_pred_wind')

# 원본 csv 파일 경로
csv_path = 'data_set/test_crop/wind_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_wind'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)
# pad_width = 3

# # 좌우로 0패딩
# padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# # 상하로 0패딩
# padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

#df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(0, n_rows-crop_size+1,7):
    for j in range(0, n_cols-crop_size+1,7):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1

## 픽셀 수만큼 crop

In [41]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_hum'):
    os.makedirs('croped_fire_pred_hum')

# 원본 csv 파일 경로
csv_path = 'data_set/crop_test/hum_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_hum'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)
pad_width = 3

# 좌우로 0패딩
padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# 상하로 0패딩
padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(n_rows-crop_size+1):
    for j in range(n_cols-crop_size+1):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1


In [42]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_rain'):
    os.makedirs('croped_fire_pred_rain')

# 원본 csv 파일 경로
csv_path = 'data_set/crop_test/rain_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_rain'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)
pad_width = 3

# 좌우로 0패딩
padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# 상하로 0패딩
padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(n_rows-crop_size+1):
    for j in range(n_cols-crop_size+1):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1


In [43]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_temp'):
    os.makedirs('croped_fire_pred_temp')

# 원본 csv 파일 경로
csv_path = 'data_set/crop_test/temp_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_temp'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)

pad_width = 3

# 좌우로 0패딩
padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# 상하로 0패딩
padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(n_rows-crop_size+1):
    for j in range(n_cols-crop_size+1):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1


In [44]:
import pandas as pd
import os

# 크롭한 csv 파일을 저장할 폴더 생성
if not os.path.exists('croped_fire_pred_wind'):
    os.makedirs('croped_fire_pred_wind')

# 원본 csv 파일 경로
csv_path = 'data_set/crop_test/wind_data0_idw.csv'

# 새로운 csv 파일을 저장할 폴더 경로
croped_csv_folder = 'croped_fire_pred_wind'

# 크롭한 csv 파일 크기 (7x7)
crop_size = 7

# 원본 csv 파일 불러오기
df = pd.read_csv(csv_path,index_col=0)

pad_width = 3

# 좌우로 0패딩
padded_lr = np.pad(df, ((0, 0), (pad_width, pad_width)), mode='constant', constant_values=32767)

# 상하로 0패딩
padded_tb = np.pad(padded_lr, ((pad_width, pad_width), (0, 0)), mode='constant', constant_values=32767)

df= pd.DataFrame(data=padded_tb)

# 데이터 프레임의 행과 열 개수
n_rows, n_cols = df.shape

# 크롭한 csv 파일의 인덱스
croped_csv_idx = 0

for i in range(n_rows-crop_size+1):
    for j in range(n_cols-crop_size+1):
        # 시작 위치부터 크롭한 데이터 가져오기
        croped_data = df.iloc[i:i+crop_size, j:j+crop_size]
        
        # 크롭한 데이터를 새로운 csv 파일로 저장
        croped_csv_path = os.path.join(croped_csv_folder, f'croped_fire_pred_{croped_csv_idx}.csv')
        croped_data.to_csv(croped_csv_path, index=True)
        
        # 새로운 csv 파일의 인덱스 업데이트
        croped_csv_idx += 1


In [32]:
#path_dir = 'data_set/csv_data'
#file_list=os.listdir(path_dir)
#for i in range(length): # 결측치를 처리한 산불의 총 갯수 lenght는 위에서 구해놨음.
#    for j in range(len(file_list)):
#        os.makedirs(f'data_set/crop_data/{file_list[j]}', exist_ok=True)
#        csv_list = sorted(glob(f"{path_dir}/{file_list[j]}/*.csv"))
#        s_index = csv_list[i].rfind('\\')
#        b_index = csv_list[i].rfind('_')
#        name=csv_list[i][s_index+5:b_index]
#        result=find_and_crop(f'data_set/fire_loc/fire_data{name}.csv',f'{path_dir}/{file_list[j]}/data{name}_idw.csv',3)
#        result=pd.DataFrame(result)
#        result.to_csv(f'data_set/crop_data/{file_list[j]}/data{name}.csv',encoding='utf-8-sig')

# crop 래스터를 png로 저장 

In [12]:
# crop되어 있는 csv를 래스터파일로 바꾼다음
# show mpa함수를 통해서 plot을 띄우고 png로 저장하는 방식을 이용 

In [28]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import os

blues = plt.cm.Blues
reds = plt.cm.Reds
greens = plt.cm.Greens
oranges = plt.cm.Oranges

new_blues = colors.LinearSegmentedColormap.from_list('new_blues', blues(np.linspace(0.01, 1.0, 256)))
new_reds = colors.LinearSegmentedColormap.from_list('new_reds', reds(np.linspace(0.01, 1.0, 256)))
new_greens = colors.LinearSegmentedColormap.from_list('new_greens', greens(np.linspace(0.01, 1.0, 256)))
new_oranges = colors.LinearSegmentedColormap.from_list('new_oranges', oranges(np.linspace(0.01, 1.0, 256)))
colormap_list=[new_greens,new_blues,new_oranges,new_reds] #각 피쳐마다 컬러맵 다르게 해야됨 
limits=[[2,100],[0,19],[-14.7,37.8],[0,20.9]] # 각 피쳐마다 범위 다르게 해야됨 (습도,강수량,기온,풍속)

In [29]:
path_dir="data_set/test_crop_7data"
file_list=os.listdir(path_dir)
file_list

['humidity', 'rainfall', 'temp', 'wind speed']

In [32]:
import natsort
from tqdm import tqdm
from memory_profiler import profile

for i in range(4):
    os.makedirs(f'data_set/test_png_7data/{file_list[i]}', exist_ok=True)
    csv_list = natsort.natsorted(glob(f"{path_dir}/{file_list[i]}/*.csv"))
    color=colormap_list[i]
    limit=limits[i]
    for j in tqdm(range(len(csv_list))):   
        
        tmp=pd.read_csv(f'{csv_list[j]}', delimiter=',', skiprows=1, encoding='utf-8')       
        data=np.array(tmp)

        # 래스터 이미지 생성
        with rasterio.open(
            'image.tif', 'w',
            driver='GTiff',
            height=6,
            width=7,  # 첫 번째 열을 제외한 폭(width)
            count=1,
            dtype='float64',
            crs='+proj=latlong',
            transform=rasterio.transform.from_bounds(0, 0, 7, 6, 7, 6)
        ) as dst:
            dst.write_band(1, data[:, 1:])  
        if j % 100 == 0: 
            #time.sleep(10)
            print(gc.collect())

        show_map('image.tif',f"data_set/test_png_7data/{file_list[i]}/data{j}", 1 , color, limit)
        
        del tmp
        del data
        del dst
        
        
os.remove(f"image.tif")

  0%|                                                                               | 3/2223 [00:08<1:21:44,  2.21s/it]

8990256


  5%|███▋                                                                           | 103/2223 [00:14<02:44, 12.88it/s]

110


  9%|███████▏                                                                       | 204/2223 [00:19<02:55, 11.47it/s]

...

 95%|█████████████████████████████████████████████████████████████████████████▋    | 2101/2223 [05:52<03:24,  1.68s/it]

110


 99%|█████████████████████████████████████████████████████████████████████████████▏| 2201/2223 [06:10<00:35,  1.60s/it]

110


100%|██████████████████████████████████████████████████████████████████████████████| 2223/2223 [06:12<00:00,  5.96it/s]


## 최적화해보자

In [6]:
import natsort

file_idx = 0  # 첫 번째 폴더만 처리
file_path = f"data_set/test_png_data/{file_list[file_idx]}"
os.makedirs(file_path, exist_ok=True)

# 래스터 이미지 생성
tif_path = 'image.tif'
csv_path_list = []
for csv_path in natsort.natsorted(glob(f"{path_dir}/{file_list[file_idx]}/*.csv")):
    csv_path_list.append(csv_path)
    if len(csv_path_list) == 1:  # 첫 번째 CSV 파일을 처리할 때만 래스터 이미지 생성
        tmp = pd.read_csv(csv_path, delimiter=',', skiprows=1, encoding='utf-8')
        data = np.array(tmp)
        height, width = data.shape
        with rasterio.open(
            tif_path, 'w',
            driver='GTiff',
            height=height,
            width=width - 1,  # 첫 번째 열을 제외한 폭(width)
            count=1,
            dtype='float64',
            crs='+proj=latlong',
            transform=rasterio.transform.from_bounds(0, 0, width-1, height, width-1, height)
        ) as dst:
            dst.write_band(1, data[:, 1:])


In [ ]:
for csv_path in csv_path_list:
    file_name = os.path.basename(csv_path)[:-4]  # 확장자 제외한 파일명 추출
    tmp = pd.read_csv(csv_path, delimiter=',', skiprows=1, encoding='utf-8')
    data = np.array(tmp)
    save_path = f"{file_path}/{file_name}.png"
    show_map(tif_path, 1.5, False, save_path, file_idx)

os.remove(tif_path)


In [7]:
# csv_list = sorted(glob(f"{path_dir}/{file_list[0]}/*.csv"))
# tmp=pd.read_csv(f'{csv_list[0]}', delimiter=',', skiprows=1, encoding='utf-8')
# data=np.array(tmp)
# print(data.shape[0])
# print(data.shape[1])

6
8


In [13]:
# 시각화할래스터, 칼럼맵,사이즈, 리턴, 경로, 경로, 피쳐넘버 
def show_map(input_raster='',save_path='', image_size=1, color='', limit=''):
    with rasterio.open(input_raster) as image_data:
        my_matrix = image_data.read(1)
        my_matrix = np.ma.masked_where(my_matrix == 32767, my_matrix)
        fig, ax = plt.subplots()
        image_hidden = ax.imshow(my_matrix, cmap=color)
        plt.close()

        fig, ax = plt.subplots()
        fig.set_facecolor("w")
        width = fig.get_size_inches()[0]
        height = fig.get_size_inches()[1]
        fig.set_size_inches(w=width, h=height)
        image = ax.imshow(my_matrix, cmap=color, vmin=limit[0], vmax=limit[1]) # 습도의 최댓값 100, 최솟값 2, 사이 간격은 256개
        plt.axis('off')
        plt.close()
        fig.savefig(f'{save_path}.png', bbox_inches='tight', transparent=True, pad_inches=0)
    del image
    del fig
    del my_matrix
    del image_hidden
    del image_data
    return 0


In [5]:
# 시각화할래스터, 칼럼맵,사이즈, 리턴, 경로, 경로, 피쳐넘버 
def rasterized(data,i,j,color,limit):
    
    with rasterio.open(
            'image.tif', 'w',
            driver='GTiff',
            height=6,
            width=7,  # 첫 번째 열을 제외한 폭(width)
            count=1,
            dtype='float64',
            crs='+proj=latlong',
            transform=rasterio.transform.from_bounds(0, 0, 7, 6, 7, 6)
        ) as dst:
            dst.write_band(1, data[:, 1:]) 
            
    #show_map('image.tif',f"data_set/test_png_data/{file_list[i]}/data{j}", 1 , color, limit)
        
    return f'image.tif'


In [109]:
import natsort
from tqdm import tqdm

raster_list=[]

for i in range(1):
    os.makedirs(f'data_set/test_png_data/{file_list[i]}', exist_ok=True)
    csv_list = natsort.natsorted(glob(f"{path_dir}/{file_list[i]}/*.csv"))
    color=colormap_list[i]
    limit=limits[i]
    for j in tqdm(range(len(csv_list))):
        tmp=pd.read_csv(f'{csv_list[j]}', delimiter=',', skiprows=1, encoding='utf-8') 
        data=np.array(tmp)
        a=rasterized(data,i,j,color,limit)
        with rasterio.open(a, 'r') as src:
            show_map(src.read(1),f"data_set/test_png_data/{file_list[i]}/data{j}", 1 , color, limit)
        del a
            
        

        
os.remove(f"image.tif")

  0%|                                                                                       | 0/111200 [00:00<?, ?it/s]


TypeError: invalid path or file: array([[32767., 32767., 32767., 32767., 32767., 32767., 32767.],
       [32767., 32767., 32767., 32767., 32767., 32767., 32767.],
       [32767., 32767., 32767., 32767., 32767., 32767., 32767.],
       [32767., 32767., 32767., 32767., 32767., 32767., 32767.],
       [32767., 32767., 32767., 32767., 32767., 32767., 32767.],
       [32767., 32767., 32767., 32767., 32767., 32767., 32767.]])

In [ ]:
len(fig_list)

In [ ]:
for i in range(40000):
    fig_list[i].savefig(f'data_set/test_png_data/humidity/test_{i}.png', bbox_inches='tight', transparent=True, pad_inches=0)

In [ ]:
for i in range(40000,80000):
    fig_list[i].savefig(f'data_set/test_png_data/humidity1/test_{i}.png', bbox_inches='tight', transparent=True, pad_inches=0)

In [ ]:
for i in range(80000,len(fig_list)):
    fig_list[i].savefig(f'data_set/test_png_data/humidity2/test_{i}.png', bbox_inches='tight', transparent=True, pad_inches=0)

최종 모델의 결과물은 data_st안에 있는 png_data에 만들어지게 됨.

In [22]:
# from PIL import Image, ImageOps

# # 병합할 4개의 이미지 파일 경로
# image_paths = ['C:/Users/ebdl/산불예측/산불예측/data_set/png_data/temp/data1.png',
#                'C:/Users/ebdl/산불예측/산불예측/data_set/png_data/humidity/data1.png',
#                'C:/Users/ebdl/산불예측/산불예측/data_set/png_data/rainfall/data1.png',
#                'C:/Users/ebdl/산불예측/산불예측/data_set/png_data/wind speed/data1.png']

# # 이미지를 열어서 리스트에 추가하고 크기를 맞춤
# images = []
# for path in image_paths:
#     img = Image.open(path).convert("RGBA")
#     img = ImageOps.fit(img, (500, 500), method=Image.LANCZOS)  # NEAREST로 설정
#     images.append(img)

# # 출력 이미지 생성
# output_width, output_height = images[0].size
# output_image = Image.new("RGBA", (output_width*2, output_height*2), (0, 0, 0, 0))

# # 이미지를 병합
# for i, img in enumerate(images):
#     x = i % 2
#     y = i // 2
#     output_image.alpha_composite(img, dest=(output_width*x, output_height*y))

# # 병합된 이미지 저장
# output_image.save("output2.png")

C:\Users\ebdl\AppData\Local\Temp\ipykernel_17476\3833523466.py:13: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = ImageOps.fit(img, (500, 500), method=Image.LANCZOS)  # NEAREST로 설정


# MERGE (이미지 병합)

In [13]:
from PIL import Image, ImageOps
import os
import re

# 이미지가 있는 폴더 경로
folder_path = 'C:/Users/ebdl/산불예측/산불예측/data_set/png_data'

# 처리할 이미지 파일들의 숫자를 저장할 리스트
image_numbers = []

# 이미지 파일의 이름에서 숫자를 추출하여 리스트에 추가
for file_name in os.listdir(folder_path + '/temp'):
    match = re.search(r'\d+', file_name)
    if match:
        number = int(match.group())
        image_numbers.append(number)

# 중복 제거 및 정렬
image_numbers = sorted(set(image_numbers))

# 이미지 파일 처리
for number in image_numbers:
    # 이미지 파일 경로 생성
    image_paths = [f'{folder_path}/temp/data{number}.png',
                   f'{folder_path}/humidity/data{number}.png',
                   f'{folder_path}/wind speed/data{number}.png',
                   f'{folder_path}/rainfall/data{number}.png']

    # 이미지를 열어서 리스트에 추가하고 크기를 맞춤
    images = []
    for path in image_paths:
        img = Image.open(path).convert("RGBA")
        img = ImageOps.fit(img, (500, 500), method=Image.LANCZOS)
        images.append(img)

    # 출력 이미지 생성
    output_width, output_height = images[0].size
    output_image = Image.new("RGBA", (output_width*2, output_height*2), (0, 0, 0, 0))

    # 이미지를 병합
    for i, img in enumerate(images):
        x = i % 2
        y = i // 2
        output_image.alpha_composite(img, dest=(output_width*x, output_height*y))

    # 병합된 이미지 저장
    output_folder = 'C:/Users/ebdl/산불예측/산불예측/data_set/png_data/merged_fire/'
    output_file = os.path.join(output_folder, f'data{number}.png')
    output_image.save(output_file)


C:\Users\ebdl\AppData\Local\Temp\ipykernel_2480\1370413513.py:33: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = ImageOps.fit(img, (500, 500), method=Image.LANCZOS)
